# GROMACS Resume -  Start a long GROMACS run that times out, and resume it

On the supercomputers, a maximum runtime of 24h is enforced. If a longer GROMACS run is needed, the run can easily be resumed. One of the outputs provided by the Rush `gmx` module is designed to be used as the input to the `gmx_resume` module, which will resume the run from the latest checkpoint. The outputs to this module are identical to those of the `gmx` module itself, so the run can be resumed as many times as necessary to finish it.

## 0.0) Imports

In [ ]:
from pathlib import Path
import rush

In [ ]:
# |hide
# Users won't generally create a workspace
# We nuke to ensure run is reproducible
import os

WORK_DIR = Path.home() / "qdx" / "tutorial-gmx-resume"

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)

In [ ]:
RUSH_TOKEN = os.getenv("RUSH_TOKEN") or "YOUR_TOKEN_HERE"
client = await rush.build_provider_with_functions(access_token=RUSH_TOKEN)

In [ ]:
# |hide
# We hide this because users will generally not set a workspace, and won't restore by default
client = await rush.build_provider_with_functions(
    batch_tags=["tutorial-resume-gmx"],
    workspace=WORK_DIR,
    restore_by_default=True,
)

## 0.1) Input Download and Selection

In [ ]:
!pdb_fetch '1B39' | pdb_selchain -A | pdb_delhetatm > '1B39_A_nohet.pdb'

## 1) Input Preparation

In [ ]:
_, prepared_protein_pdb = await client.prepare_protein(
    Path.cwd() / "1B39_A_nohet.pdb"
)

## 2.1) Run GROMACS (modules: gmx, gmx_resume)
Next we will run a molecular dynamics simulation on our protein using gromacs via the `gmx` module.

We'll set `timeout_duration_mins = 1` so that the run times out before it finishes, and then resume via the `gmx_resume` module using the first output, which is the archive that contains all the necessary data for resuming the run from the last saved checkpoint.

We'll set `checkpoint_interval_mins = 1.0/60` so that the checkpointing takes place once per second.

We can restart as many times as we need to in order for the run to finish. Use a unique tag for each restarted run so that each sequential restart will be tagged and cached appropriately. See below for an example.

For each restarted run, please pass the same config file that was passed to the original `gmx` call. There is no support for shortening or extending runs, or changing any other run parameters, of runs that have already been started. Passing the same config ensures that progress is reported properly and that there are no other inconsistencies. So, the initial run config should specify the full desired run.

One current limitation is that the frame selection can only operate on the data generated by the last call to `gmx` or `gmx_restart`. Otherwise, the output xtc files from all the calls must be joined and processed manually.

In [ ]:
gmx_config = {
    "params_overrides": {
        "nvt": {"nsteps": 2000},
        "npt": {"nsteps": 2000},
        "md": {"nsteps": 150000},
    },
    "frame_sel": {
        "start_time_ps": 290,
        "end_time_ps": 300,
        "delta_time_ps": 1,
    },
    "checkpoint_interval_mins": 1.0 / 60,
    "timeout_duration_mins": 1,
    "num_gpus": 1,
    "save_wets": False,
}

In [ ]:
resume_files_first, streaming_outputs, static_outputs, *rest = await client.gmx(
    None,
    prepared_protein_pdb,
    None,
    gmx_config,
    resources={"gpus": 1, "storage": 1, "storage_units": "GB"},
)


In [ ]:
# Check the progress and repeat this call as needed,
# incrementing the integer in the tag with each subsequent call
resume_files_second, _, _, xtc_dry, pdb_dry, _, _ = await client.gmx_resume(
    resume_files_first,
    gmx_config,
    tags = ["gmx-resume-1"],
    resources={"gpus": 1, "storage": 1, "storage_units": "GB"},
)

In [ ]:
resume_files_third, _, _, xtc_dry, pdb_dry, _, _ = await client.gmx_resume(
    resume_files_second,
    gmx_config,
    tags = ["gmx-resume-2"],
    resources={"gpus": 1, "storage": 1, "storage_units": "GB"},
)

In [ ]:
await pdb_dry.get()

## Checking progress of GROMACS run
To determine if your GROMACS run is done, or if further runs are required, you can look at the progress output.
How to check this output is demonstrated below. 


This is an example progress event that indicates that the job has not completed, and will require resuming.
`n` is the number of execution steps in the GMX module. If `n` is less than `n_expected`, or `done` is false the run is not yet completed.


`gmx_resume_tengu progress: {
  "n": 600,
  "n_expected": 601,
  "n_max": 601,
  "done": false
}`

In [ ]:
# Checking 
result1 = (await client.module_instance(resume_files_first.source))
result2 = (await client.module_instance(resume_files_second.source))
result3 = (await client.module_instance(resume_files_third.source))

In [ ]:
print(result1.progress)
print(result2.progress)
print(result3.progress)